In [2]:
# region General Imports
import os
import uuid
import shutil
import time
import random
import datetime
import glob
import pickle
import tqdm
import copy
import optuna
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import mne
from rich import print as rprint
from rich.pretty import pprint as rpprint
from tqdm import tqdm
from itertools import chain
from functools import partial
# endregion General Imports

from model_optim.model_optimizer import ModelOptimizer

# Dataset
from custom_datasets.fatigue_mi import FatigueMI

%load_ext autoreload
%autoreload 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# np.load("./bkup/best_subjects_search_results/subject_6_shallow_conv_net_7b71b3b6-52f7-435c-be85-2cb99e793f98.npy", allow_pickle=True).item().user_attrs['trial_data']

prev_best_full = np.load("./_temp/[1]/292cbc92b8cf46da9986fe7d8447819f/model/study_best_trial.npy", allow_pickle=True).item()
# prev_best_full = np.load("./_temp/[2]/6405d11e654b42aca9df48458c67ecde/model/study_best_trial.npy", allow_pickle=True).item()
# prev_best_full = np.load("./_temp/[3]/9c1b753483db409a90eab7b7149b8af8/model/study_best_trial.npy", allow_pickle=True).item()
prev_best = prev_best_full.user_attrs['trial_data']

rpprint({
    "sfreq": prev_best_full.params["sfreq"],
    "batch_size": prev_best_full.params["batch_size"],
    "val_acc": np.max(prev_best["val_accuracy"]),
    "test_acc": prev_best["test_accuracy"],
    "channels": prev_best["channels_selected"],
})

{
│   'sfreq': 300,
│   'batch_size': 160,
│   'val_acc': 0.8333333134651184,
│   'test_acc': 0.8636363744735718,
│   'channels': array(['Fz', 'C4', 'Fp2', 'T5', 'O2', 'F7', 'F8', 'A2', 'T6', 'T4'],
│     dtype='<U3')
}

In [4]:
model_optimizer = ModelOptimizer(
    dataset=FatigueMI(),
    model_name="shallow_conv_net"
)

In [5]:
study = model_optimizer.search_best_model(
    subjects=[3],
    max_iter = 25,
    max_epochs=5,
    max_stag_count=10,
    rounds=1,
    replace_previous_study_for_subjects=True
)

  0%|          | 0/25 [00:00<?, ?it/s]

Adding metadata with 3 columns


2024-03-07 20:07:55.496698: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 20:07:55.520511: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 20:07:55.520576: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 20:07:55.523689: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 20:07:55.523821: I external/local_xla/xla/stream_executor

Epoch 1/5


2024-03-07 20:07:58.409338: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-03-07 20:07:59.534754: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-03-07 20:08:00.422877: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-07 20:08:00.863993: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-07 20:08:02.066783: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f2f4fe50cf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-07 20:08:02.066861: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, C

...

Best trial: 2. Best value: 4.47435:  44%|████▍     | 11/25 [00:49<00:59,  4.26s/it]



Adding metadata with 3 columns
Epoch 1/5
1/1 [==============================] - ETA: 0s - loss: 50.2647 - accuracy: 0.4412
Epoch 00001: val_accuracy improved from -inf to 0.72222, storing weights.
1/1 [==============================] - 2s 2s/step - loss: 50.2647 - accuracy: 0.4412 - val_loss: 40.7313 - val_accuracy: 0.7222 - lr: 0.0010
Epoch 2/5
1/1 [==============================] - ETA: 0s - loss: 41.9142 - accuracy: 0.4559
Epoch 00002: val_accuracy did not improve
1/1 [==============================] - 0s 54ms/step - loss: 41.9142 - accuracy: 0.4559 - val_loss: 41.1668 - val_accuracy: 0.1667 - lr: 0.0010
Epoch 3/5
1/1 [==============================] - ETA: 0s - loss: 42.4546 - accuracy: 0.5294
Epoch 00003: val_accuracy did not improve
1/1 [==============================] - 0s 54ms/step - loss: 42.4546 - accuracy: 0.5294 - val_loss: 32.7816 - val_accuracy: 0.3889 - lr: 0.0010
Epoch 4/5
1/1 [==============================] - ETA: 0s - loss: 32.5851 - accuracy: 0.5441
Epoch 00004: v

Best trial: 2. Best value: 4.47435:  48%|████▊     | 12/25 [00:52<00:48,  3.75s/it]

Sampling frequency of the instance is already 300.0, returning unmodified.
Adding metadata with 3 columns
Epoch 1/5
2/2 [==============================] - ETA: 0s - loss: 24.3164 - accuracy: 0.5000
Epoch 00001: val_accuracy improved from -inf to 0.27778, storing weights.
2/2 [==============================] - 2s 560ms/step - loss: 24.3164 - accuracy: 0.5000 - val_loss: 22.1662 - val_accuracy: 0.2778 - lr: 0.0010
Epoch 2/5
1/2 [==============>...............] - ETA: 0s - loss: 27.1406 - accuracy: 0.5000
Epoch 00002: val_accuracy improved from 0.27778 to 0.72222, storing weights.
2/2 [==============================] - 0s 51ms/step - loss: 26.6745 - accuracy: 0.4853 - val_loss: 12.9897 - val_accuracy: 0.7222 - lr: 0.0010
Epoch 3/5
1/2 [==============>...............] - ETA: 0s - loss: 14.6628 - accuracy: 0.5000
Epoch 00003: val_accuracy did not improve
2/2 [==============================] - 0s 49ms/step - loss: 15.0385 - accuracy: 0.5147 - val_loss: 12.6731 - val_accuracy: 0.3889 - lr: 0.

Best trial: 2. Best value: 4.47435:  52%|█████▏    | 13/25 [00:55<00:51,  4.28s/it]

In [26]:
model_optimizer.get_study_metrics(study).sort_values(by="scores", ascending=True)
# model_optimizer.get_study_metrics(study).sort_values(by="test_acc", ascending=False)

,train_acc,test_acc,val_acc,train_val_acc_diff,train_loss,val_loss,train_val_loss_diff,test_loss,scores,channels_selected,sfreq,batch_size
2,0.808824,0.545455,0.722222,0.086601,5.571209,4.473697,1.097512,5.575230,4.474347,"[C3, F3, Fz, F4, P4, Cz, Pz, Fp1, Fp2, T5, F8,...",128,64
5,0.573529,0.500000,0.833333,0.259804,7.480617,7.187803,0.292814,8.553566,8.188353,"[P3, F3, F4, Cz, Pz, Fp1, T3, O1, F7, F8, T4]",128,256
6,0.632353,0.500000,0.611111,0.021242,15.483885,11.079287,4.404598,30.025928,12.079887,"[P3, C3, F3, F4, C4, P4, Pz, T5, O2, F7, F8, T4]",256,32
12,0.558824,0.590909,0.722222,0.163399,12.344352,12.673098,0.328746,13.615680,14.207707,"[C3, F3, Fz, P4, Fp1, Fp2, T5, O1, F7, F8, A2,...",300,64
10,0.544118,0.545455,0.777778,0.233660,23.069323,17.256979,5.812344,60.630184,18.257579,"[F3, F4, C4, Fp1, Fp2, T3, O1, O2, F8, A2, T6,...",300,192
0,0.632353,0.454545,0.555556,0.076797,27.278645,25.592693,1.685951,46.432106,26.593093,"[C3, F3, Fz, C4, P4, Fp1, T3, O1]",128,64
11,0.544118,0.545455,0.722222,0.178105,29.934460,29.465427,0.469032,41.052807,30.465777,"[C3, Fz, F4, Fp1, O2, F8, T6]",128,256
8,0.544118,0.454545,0.833333,0.289216,37.504967,33.852737,3.652229,46.772701,34.853537,"[C3, F3, Fz, C4, P4, Cz, Pz, Fp1, Fp2, O1, O2,...",128,128
9,0.558824,0.545455,0.611111,0.052288,42.619907,35.933231,6.686676,58.202469,36.933781,"[Fz, P4, Pz, Fp1, Fp2, T3, O1, O2, A2, T6, T4]",128,128
7,0.588235,0.500000,0.777778,0.189542,42.740204,36.299633,6.440571,73.249672,37.300083,"[Fz, C4, Pz, Fp1, Fp2, T3, F7, A2, T6]",128,64


In [23]:
rpprint({ k: v for k, v in study.best_trial.params.items() if not k.startswith("channels") })
rprint("test_accuracy =", study.best_trial.user_attrs["trial_data"]["test_accuracy"])
rprint("val_accuracy =", np.max(study.best_trial.user_attrs["trial_data"]["val_accuracy"]))
rprint("channels_selected =", study.best_trial.user_attrs["trial_data"]["channels_selected"])

{
│   'sfreq': 128,
│   'batch_size': 64,
│   'pool_size_d2': 35,
│   'strides_d2': 31,
│   'conv_filters_d2': 52,
│   'conv2d_1_units': 190,
│   'conv2d_2_units': 160,
│   'l2_reg_1': 0.07525442044886897,
│   'l2_reg_2': 0.03472242296177845,
│   'l2_reg_3': 0.8819445565677363,
│   'dropout_rate': 0.2
}

test_accuracy = 0.5454545617103577

val_accuracy = 0.7222222089767456

channels_selected = ['C3' 'F3' 'Fz' 'F4' 'P4' 'Cz' 'Pz' 'Fp1' 'Fp2' 'T5' 'F8' 'T6' 'T4']

In [20]:
# model_optimizer.clean_up(
#     action="remove_all_but_best_trial_data",
#     subjects=[1],
# )